In [2]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116


In [3]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [6]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [7]:
result = model(tokens)

In [8]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [9]:
int(torch.argmax(result.logits))+1

4

In [37]:
r = requests.get('https://www.mouthshut.com/product-reviews/Amazon-com-reviews-925000493')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [38]:
soup.find

<bound method Tag.find of 
<!DOCTYPE HTML>

<html lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head id="ctl00_ctl00_Head1"><meta content="P6Kqf7KhXgEvVAg-ZcEtxTLDJ-aXEVi7NsRPIDpgE2Q" name="google-site-verification"/><title>
	AMAZON.COM Reviews, Feedback, Complaint, Experience, Customer Care Number - MouthShut.com
</title>
<style>
        .surveylayer {
            position: fixed;
            z-index: 1111;
            background: #fff;
            left: 34%;
            top: -581px;
            border-radius: 5px;
        }

            .surveylayer section {
                margin: 0;
                padding: 1px;
            }

        .surveylayerclose {
            position: absolute;
            top: 11px;
            right: 11px;
            font-weight: 900 !important;
            /* background-color: #6f9d20; */
            padding: 10px;
            border-radius: 50%;
            color: #666;
    

In [39]:
reviews

[]

In [40]:
import numpy as np
import pandas as pd

In [41]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [42]:
results

[]

In [43]:
results[0].text

IndexError: list index out of range

In [44]:
df['review'].iloc[0]

IndexError: single positional indexer is out-of-bounds